In [1]:
import pandas as pd
import numpy as np
import time
import os
import glob
import json
from collections import defaultdict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

## Configuration

In [2]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../../results'
# ID of the training and test data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0005_19042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0005_19042021


## Model

In [ ]:
results_ls = []

In [7]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

print('Train -> X: {}, Y: {}'.format(X_train.shape, Y_train.shape))
print('Test -> X: {}, Y: {}'.format(X_test.shape, Y_test.shape))

Train -> X: (943372, 15), Y: (943372, 24)
Test -> X: (161200, 15), Y: (161200, 24)


In [8]:
# train_samples = X_train.shape[0]
# step = int(train_samples / 100000)
# idx = list(range(0, train_samples, step))
# X_train = X_train[idx, :]
# Y_train = Y_train[idx, :]

# print('Train -> X: {}, Y: {}'.format(X_train.shape, Y_train.shape))
# print('Test -> X: {}, Y: {}'.format(X_test.shape, Y_test.shape))

Train -> X: (104820, 15), Y: (104820, 24)
Test -> X: (161200, 15), Y: (161200, 24)


In [9]:
params = {
    'objective': 'reg:squarederror',
    'booster': 'gbtree', 
    'eta': 0.3, 
#     'gamma': 0.5, 
    'max_depth': 6,
    'subsample': 0.75,
#     'lambda': 0.8,
    'nthread': 8,
    'seed': 0
}

results = defaultdict(list)

tr_time = []
for target in range(Y_train.shape[1]):

    dtrain = xgb.DMatrix(data=X_train, label=Y_train[:, target])
    dtest = xgb.DMatrix(data=X_test, label=Y_test[:, target])

    callbacks = [xgb.callback.EarlyStopping(rounds=50, metric_name='rmse', maximize=False, save_best=True)]
    
    t_start = time.time()
    model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dtest, 'rmse')], callbacks=callbacks, verbose_eval=False)
    tr_time.append(time.time() - t_start)
    
    train_preds = model.predict(dtrain)
    test_preds = model.predict(dtest)

    results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test_MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test_MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test_R2'].append(r2_score(Y_test[:, target], test_preds))
    
print('Training time: {:.4f}'.format(sum(tr_time)))

results_summary = {}
results_summary['Training time'] = sum(tr_time)
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results['_'.join([subset, loss])][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))
            
results_ls.append(results_summary)

/home/hl0d0w1g/data/Educación/Master Ing de la Informacion para la Salud/Master Thesis/EXOSAFE-ML/venv/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/hl0d0w1g/data/Educación/Master Ing de la Informacion para la Salud/Master Thesis/EXOSAFE-ML/venv/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/hl0d0w1g/data/Educación/Master Ing de la Informacion para la Salud/Master Thesis/EXOSAFE-ML/venv/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/hl0d0w1g/data/Educación/Master Ing de la Informacion para la Sal

Training time: 161.9432
Train Fx MAE: 6.5724 ± 2.0666
Train Fx MSE: 93.3628 ± 52.7520
Train Fx R2: 0.6768 ± 0.1153
Train Fy MAE: 7.1481 ± 4.1721
Train Fy MSE: 128.9137 ± 122.1639
Train Fy R2: 0.4441 ± 0.1707
Train Fz MAE: 9.4273 ± 2.6308
Train Fz MSE: 179.2909 ± 93.4021
Train Fz R2: 0.6923 ± 0.1257
Test Fx MAE: 10.9019 ± 3.8761
Test Fx MSE: 257.3233 ± 175.3165
Test Fx R2: 0.2184 ± 0.1608
Test Fy MAE: 9.9425 ± 6.5834
Test Fy MSE: 321.1926 ± 391.1755
Test Fy R2: 0.0735 ± 0.1677
Test Fz MAE: 16.3788 ± 7.1936
Test Fz MSE: 644.5339 ± 641.5028
Test Fz R2: 0.1875 ± 0.1511


In [11]:
pd.DataFrame(results_ls)

,Training time,Train Fx MAE,Train Fx MSE,Train Fx R2,Train Fy MAE,Train Fy MSE,Train Fy R2,Train Fz MAE,Train Fz MSE,Train Fz R2,Test Fx MAE,Test Fx MSE,Test Fx R2,Test Fy MAE,Test Fy MSE,Test Fy R2,Test Fz MAE,Test Fz MSE,Test Fz R2
0,161.943248,6.5724 ± 2.0666,93.3628 ± 52.7520,0.6768 ± 0.1153,7.1481 ± 4.1721,128.9137 ± 122.1639,0.4441 ± 0.1707,9.4273 ± 2.6308,179.2909 ± 93.4021,0.6923 ± 0.1257,10.9019 ± 3.8761,257.3233 ± 175.3165,0.2184 ± 0.1608,9.9425 ± 6.5834,321.1926 ± 391.1755,0.0735 ± 0.1677,16.3788 ± 7.1936,644.5339 ± 641.5028,0.1875 ± 0.1511
